<a href="https://colab.research.google.com/github/sutummala/AutismNet/blob/main/Autism_patchNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
#import tensorflow_addons as tfa
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
import sklearn
import nibabel as nib
import numpy as np
import random
import matplotlib.pyplot as plt
#from pytorch_grad_cam import GradCAM

In [ ]:
healthy_data = np.load('/content/drive/My Drive/Datasets/Autism/healthy_patches_2.npy')
#healthy_data_2 = np.load('/content/drive/My Drive/Datasets/Autism/healthy_patches_2.npy')
#healthy_data = np.concatenate((healthy_data_1, healthy_data_2))
healthy_labels = np.zeros(np.shape(healthy_data)[0])

autism_data = np.load('/content/drive/My Drive/Datasets/Autism/autism_patches_2.npy')
#autism_data_2 = np.load('/content/drive/My Drive/Datasets/Autism/autism_patches_2.npy')
#autism_data = np.concatenate((autism_data_1, autism_data_2))
autism_labels = np.ones(np.shape(autism_data)[0])

index = int(0.8 * healthy_data.shape[0])

X = np.concatenate((healthy_data[:index], autism_data[:index])) * 255
X_train = np.expand_dims(X, axis = -1)
healthy_test = np.expand_dims(healthy_data[index:], axis = -1) 
autism_test = np.expand_dims(autism_data[index:], axis = -1)

y_train = np.concatenate((healthy_labels[:index], autism_labels[:index]))
healthy_test_labels = healthy_labels[index:]
autism_test_labels = autism_labels[index:]

X_test = np.concatenate((healthy_test, autism_test))
y_test = np.concatenate((healthy_test_labels, autism_test_labels))

print(f'no.of healthy patches for testing are {np.shape(healthy_test_labels)[0]}')
print(f'no.of diseased patches for testing are {np.shape(autism_test_labels)[0]}')

In [ ]:
print(f'no.of healthy patches for testing are {np.shape(X_test)[0]}')
print(f'no.of diseased patches for testing are {np.shape(y_test)[0]}')

In [ ]:
folds = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 1)

#X_train = np.expand_dims(X_train, axis = -1) * 255 # expanding dimensions

for train_index, test_index in folds.split(X_train, y_train):
    input_cv, input_test, targets_cv, targets_test = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]

print(f'shape of input for CV is {input_cv.shape}')
print(f'input size for cross-validation is {len(targets_cv)}')
print(f'no.of healthy in CV are {np.shape(np.nonzero(targets_cv))[1]}')

#left_input_test = left_input[cv_index:]
#right_input_test = right_input[cv_index:]
#targets_test = targets[cv_index:]

print(f'shape of input for testing is {input_test.shape}')
print(f'input size for testing is {len(targets_test)}')
print(f'no.of healthy in test are {np.shape(np.nonzero(targets_test))[1]}')

In [ ]:
img_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3], 1, )
print(img_shape)

In [ ]:
model = tf.keras.Sequential(name = 'patchNet')

model.add(tf.keras.layers.Conv3D(8, (3, 3, 3), input_shape = img_shape, strides = (1,1,1), padding = 'same', kernel_regularizer = 'L2', name = 'conv3d_1'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SpatialDropout3D(0.1))

model.add(tf.keras.layers.Conv3D(16, (3, 3, 3), strides = (1,1,1), padding = 'same', kernel_regularizer = 'L2', name = 'conv3d_2'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling3D(strides = (2, 2, 2)))
model.add(tf.keras.layers.SpatialDropout3D(0.1))

model.add(tf.keras.layers.Conv3D(32, (3, 3, 3), strides = (1,1,1), padding = 'same', kernel_regularizer = 'L2', name = 'conv3d_3'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling3D(strides = (2, 2, 2)))
model.add(tf.keras.layers.SpatialDropout3D(0.1))

model.add(tf.keras.layers.Conv3D(64, (3, 3, 3), strides = (1,1,1), padding = 'same', kernel_regularizer = 'L2', name = 'conv3d_4'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling3D(strides = (2, 2, 2)))
#model.add(tf.keras.layers.SpatialDropout3D(0.1))

model.add(tf.keras.layers.Conv3D(128, (3, 3, 3), strides = (1,1,1), padding = 'same', kernel_regularizer = 'L2', name = 'conv3d_5'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling3D(strides = (2, 2, 2)))
#model.add(tf.keras.layers.SpatialDropout3D(0.1))

model.add(tf.keras.layers.GlobalAveragePooling3D())
model.add(tf.keras.layers.Dense(256, kernel_regularizer = 'L2', activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(256, kernel_regularizer = 'L2', activation = 'relu'))
#model.add(tf.keras.layers.GaussianNoise(0.1))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(256, kernel_regularizer = 'L2', activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

#patchNet_model = tf.keras.Model(inputs = input, outputs =  output, name = 'patchNet')

model.summary()


In [ ]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.0005), loss = 'binary_crossentropy', metrics = 'accuracy')
    
no_of_epochs = 200

history_fine = model.fit(input_cv, targets_cv, batch_size = 32,
                              epochs = no_of_epochs,
                              shuffle = True
                              validation_data = (input_test, targets_test))

In [ ]:
predicton_prob = model.predict(X_test)

print(np.transpose(predicton_prob))

predicton_prob[predicton_prob > 0.5] = 1
predicton_prob[predicton_prob <= 0.5] = 0

tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, predicton_prob).ravel()

targets_test = y_test

print(f'test Accuracy: {sklearn.metrics.accuracy_score(targets_test, predicton_prob)}')
print(f'test ROC (AUC): {sklearn.metrics.roc_auc_score(targets_test, predicton_prob)}')
print(f'test Sensitivity (Recall): {sklearn.metrics.recall_score(targets_test, predicton_prob)}')
print(f'test Precision: {sklearn.metrics.precision_score(targets_test, predicton_prob)}')
print(f'test F1-score: {sklearn.metrics.f1_score(targets_test, predicton_prob)}')
print(f'test Mathews Correlation Coefficient: {sklearn.metrics.matthews_corrcoef(targets_test, predicton_prob)}')
print(f'test Specificity: {tn/(tn+fp)}')
print(targets_test)